# DoorDash Delivery Duration Prediction

When a consumer places an order on DoorDash, we show the expected time of delivery. It is very important for DoorDash to get this right, as it has a big impact on consumer experience. In this project we will be building a model to predict the estimated time taken for a delivery. 

Concretely, for a given delivery you must predict the total delivery duration seconds, i.e. the time taken from
- Start: the time when the consumer submits the order (`created_at`) to...
- End: when the order will be delivered to the consumer (`actual_delivery_time`)

## Data Exploration

We'll start with a classic exploration of the data to get familiar with what we're working with, and then we'll continue on to investigate the features needing some processing.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)

In [3]:
historical_data = pd.read_csv("data/historical_data.csv")
historical_data.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


In [4]:
historical_data.shape

(197428, 16)

In [5]:
historical_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     196441 non-null  float64
 1   created_at                                    197428 non-null  object 
 2   actual_delivery_time                          197421 non-null  object 
 3   store_id                                      197428 non-null  int64  
 4   store_primary_category                        192668 non-null  object 
 5   order_protocol                                196433 non-null  float64
 6   total_items                                   197428 non-null  int64  
 7   subtotal                                      197428 non-null  int64  
 8   num_distinct_items                            197428 non-null  int64  
 9   min_item_price                                19

We see some id columns in the DataFrame, which are generally not useful in prediction tasks. The dataset includes null values which we will deal with at some point within data preprocessing. Most importantly, there is no column in the dataset that represents our target value (the value that we will predict) explicitly. We can deduce it by using the difference between `created_at` and `actual_delivery_time`. To do that, we first convert the columns to the DateTime data type. Then, we obtain the time in terms of seconds through `_.dt.totalseconds( )`. We can use the calculated value as our target in regression.

In [6]:
from datetime import datetime

historical_data['created_at'] = pd.to_datetime(historical_data['created_at'])
historical_data['actual_delivery_time'] = pd.to_datetime(historical_data['actual_delivery_time'])
historical_data['actual_total_delviery_duration'] = (historical_data['actual_delivery_time'] - historical_data['created_at']).dt.total_seconds()
historical_data.sample(3)

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,actual_total_delviery_duration
175813,2.0,2015-01-24 18:23:56,2015-01-24 19:24:39,3955,smoothie,2.0,2,1944,2,595,1199,12.0,36.0,14.0,251,495.0,3643.0
79762,2.0,2015-01-22 19:02:58,2015-01-22 19:25:44,5704,vietnamese,5.0,2,2098,2,836,1138,31.0,36.0,28.0,251,196.0,1366.0
80915,4.0,2015-02-01 20:39:21,2015-02-01 21:19:24,6323,vietnamese,2.0,3,3135,3,695,1395,35.0,35.0,48.0,251,501.0,2403.0


Another feature that we can add to the dataframe is estimated non-preparation duration. We have estimated store-to-consumer driving and estimated order place durations. Total order delivery duration could be divided into two main operations: Preparation and non-preparation. It could be a useful feature. We will see it later. Currently, we are looking for what we can extract from the given data. This is the feature engineering part.

In [7]:
historical_data['estimated_non_prep_duration'] = historical_data['estimated_store_to_consumer_driving_duration'] + historical_data['estimated_order_place_duration']
historical_data.sample(3)

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,actual_total_delviery_duration,estimated_non_prep_duration
51988,2.0,2015-01-24 22:05:02,2015-01-24 22:45:57,667,mediterranean,5.0,5,3675,4,595,995,30.0,61.0,48.0,251,705.0,2455.0,956.0
48127,4.0,2015-02-07 02:42:53,2015-02-07 04:24:24,422,indian,3.0,5,4675,4,200,1395,84.0,84.0,151.0,251,530.0,6091.0,781.0
68477,2.0,2015-02-17 20:34:06,2015-02-17 21:08:04,4210,mexican,2.0,8,5205,7,195,1195,57.0,49.0,59.0,251,148.0,2038.0,399.0


We have total busy dashers and total onshift dashers as two columns. This columns could be informative about current availability. We can see how busy the operation is calculating a busy dashers ratio with dividing `total_busy_dashers` to `total_onshift_dashers`.

In [8]:
historical_data['busy_dashers_ratio'] = historical_data['total_busy_dashers'] / historical_data['total_onshift_dashers']
historical_data.sample(7)

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,actual_total_delviery_duration,estimated_non_prep_duration,busy_dashers_ratio
172590,6.0,2015-02-14 02:43:25,2015-02-14 03:26:41,6111,breakfast,5.0,2,1625,2,975,1225,NaN,NaN,NaN,251,855.0,2596.0,1106.0,NaN
183883,2.0,2015-01-28 02:29:05,2015-01-28 03:12:36,2130,chinese,3.0,3,2050,3,550,950,83.0,61.0,95.0,251,759.0,2611.0,1010.0,0.734940
4827,3.0,2015-01-26 03:23:14,2015-01-26 03:59:33,4534,american,1.0,2,3750,2,1475,1550,11.0,3.0,2.0,446,268.0,2179.0,714.0,0.272727
18296,2.0,2015-02-06 05:34:36,2015-02-06 06:16:37,6566,cafe,4.0,4,1560,3,175,395,30.0,32.0,45.0,251,622.0,2521.0,873.0,1.066667
80879,2.0,2015-02-12 18:59:47,2015-02-12 19:32:41,6904,american,2.0,1,1188,1,1188,1188,30.0,30.0,41.0,251,474.0,1974.0,725.0,1.000000
3540,1.0,2015-02-08 01:57:41,2015-02-08 03:06:29,2938,pizza,1.0,2,2430,2,1035,1395,21.0,22.0,38.0,446,763.0,4128.0,1209.0,1.047619
69017,2.0,2015-02-15 02:38:40,2015-02-15 04:07:19,6145,mexican,3.0,4,2396,4,299,1399,51.0,49.0,94.0,251,466.0,5319.0,717.0,0.960784


We can see that there are ratios above 1, which is interesting.

In [9]:
historical_data[historical_data['busy_dashers_ratio'] > 1]

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,actual_total_delviery_duration,estimated_non_prep_duration,busy_dashers_ratio
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,4024.0,1136.0,2.000000
7,3.0,2015-02-12 03:03:35,2015-02-12 03:36:20,5477,NaN,1.0,4,4850,4,750,1800,7.0,8.0,7.0,446,626.0,1965.0,1072.0,1.142857
14,1.0,2015-02-12 03:36:46,2015-02-12 04:14:39,2841,italian,1.0,1,1525,1,1525,1525,5.0,6.0,8.0,446,795.0,2273.0,1241.0,1.200000
18,1.0,2015-01-31 04:35:54,2015-01-31 05:47:30,2841,italian,1.0,2,3150,2,1425,1725,4.0,9.0,12.0,446,548.0,4296.0,994.0,2.250000
20,1.0,2015-01-31 23:45:12,2015-02-01 00:14:05,4139,mexican,1.0,5,1285,3,150,400,12.0,13.0,11.0,446,424.0,1733.0,870.0,1.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197411,1.0,2015-02-02 17:50:23,2015-02-02 18:12:57,2956,fast,4.0,2,1297,2,369,639,5.0,6.0,7.0,251,186.0,1354.0,437.0,1.200000
197412,1.0,2015-02-10 23:43:08,2015-02-11 00:47:24,2956,fast,4.0,1,674,1,345,345,10.0,11.0,12.0,251,706.0,3856.0,957.0,1.100000
197418,1.0,2015-01-30 20:50:23,2015-01-30 22:24:38,2956,fast,4.0,2,1528,2,639,729,26.0,29.0,34.0,251,791.0,5655.0,1042.0,1.115385
197422,1.0,2015-01-31 19:48:15,2015-01-31 20:27:39,2956,fast,4.0,7,2445,3,145,585,23.0,24.0,24.0,251,608.0,2364.0,859.0,1.043478


Let's look at the ids now. We have two columns, `market_id` and `store_id` that include ids. We can incorporate them by hot-encoding to the DataFrame and dropping the original column. However, we are not sure how many columns they would add to the DataFrame in a hot-encoding case. Let's see.

In [10]:
historical_data['market_id'].nunique()

6

In [11]:
historical_data['store_id'].nunique()

6743

Definitely not going to one-hot encode the `store_id` as that would add an insufferable amount of columns, so we'll continue with just `market_id` and drop `store_id`. There is also `order_protocol` column in our dataset as a categorical variable. And, order protocol might have some effect on delivery speed. It's just a guess but we would see the effect in feature importances check part. Let's decide to hot-encode and keep this column in later.

In [12]:
historical_data['order_protocol'].nunique()

7

Another categorical feature is `store_primary_category`. This column includes some nulls and actually, we can fill it. We can use the most repeated primary category with the associated store to fill this store's null primary category column.

In [13]:
store_id_unique = historical_data['store_id'].unique().tolist()
store_id_and_category = {store_id: historical_data[historical_data.store_id == store_id].store_primary_category.mode() for store_id in store_id_unique}

In [14]:
def fill(store_id):
    """Return primary store catoegory from dictionary"""
    try:
        return store_id_and_category[store_id].values[0]
    except:
        return np.nan

# fill null values
historical_data['nan_free_store_primary_category'] = historical_data.store_id.apply(fill)

In [15]:
historical_data.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,actual_total_delviery_duration,estimated_non_prep_duration,busy_dashers_ratio,nan_free_store_primary_category
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,3779.0,1307.0,0.424242,american
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,4024.0,1136.0,2.000000,indian
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0,1781.0,1136.0,0.000000,indian
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0,3075.0,735.0,1.000000,indian
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0,2390.0,1096.0,1.000000,indian


Now, we see nan values in row 2,3,4 are gone. However, we observe also there are discrepancies between the column that we created and the original primary categories. For example, store 5477 is associated with Mexican food in the original and with Indian in our new column. Let's check why it's happening.

In [16]:
store_id_and_category[5477]

0     indian
1    mexican
Name: store_primary_category, dtype: object

Our dictionary says for the given store id, Mexican and Indian are seen equally on the DataFrame. And, it chooses the first value as in the fill function to replace nan values. Let's confirm it from the original dataset.

In [17]:
historical_data[historical_data.store_id == 5477]

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,actual_total_delviery_duration,estimated_non_prep_duration,busy_dashers_ratio,nan_free_store_primary_category
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,4024.0,1136.0,2.000000,indian
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0,1781.0,1136.0,0.000000,indian
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0,3075.0,735.0,1.000000,indian
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0,2390.0,1096.0,1.000000,indian
5,3.0,2015-01-28 20:30:38,2015-01-28 21:08:58,5477,NaN,1.0,3,5000,3,1500,1900,2.0,2.0,2.0,446,338.0,2300.0,784.0,1.000000,indian
6,3.0,2015-01-31 02:16:36,2015-01-31 02:43:00,5477,NaN,1.0,2,3900,2,1200,2700,10.0,9.0,9.0,446,638.0,1584.0,1084.0,0.900000,indian
7,3.0,2015-02-12 03:03:35,2015-02-12 03:36:20,5477,NaN,1.0,4,4850,4,750,1800,7.0,8.0,7.0,446,626.0,1965.0,1072.0,1.142857,indian
8,2.0,2015-02-16 00:11:35,2015-02-16 00:38:01,5477,indian,3.0,4,4771,3,820,1604,8.0,6.0,18.0,446,289.0,1586.0,735.0,0.750000,indian
9,3.0,2015-02-18 01:15:45,2015-02-18 02:08:57,5477,NaN,1.0,2,2100,2,700,1200,2.0,2.0,2.0,446,715.0,3192.0,1161.0,1.000000,indian
10,3.0,2015-02-02 19:22:53,2015-02-02 20:09:19,5477,NaN,4.0,4,4300,4,1200,1500,1.0,1.0,1.0,446,453.0,2786.0,899.0,1.000000,indian


Yes, we were right. Mexican and Indian both are given in one instance for store 5477. Hence, we can continue with the next steps of feature engineering. We can't use these categories with their original names. We will apply one-hot encoding on them. Since the cuisine might cause changes in preparation duration, we don't want to drop this column. We decided previously to apply one-hot encoding on some of the other categorical variables too. Let's do it by using `_pandas.getdummies( )`.

In [19]:
# create dummies for order protocol
order_protocol_dummies = pd.get_dummies(historical_data.order_protocol)
order_protocol_dummies = order_protocol_dummies.add_prefix('order_protocol_')
order_protocol_dummies.head()

,order_protocol_1.0,order_protocol_2.0,order_protocol_3.0,order_protocol_4.0,order_protocol_5.0,order_protocol_6.0,order_protocol_7.0
0,True,False,False,False,False,False,False
1,False,True,False,False,False,False,False
2,True,False,False,False,False,False,False
3,True,False,False,False,False,False,False
4,True,False,False,False,False,False,False


In [20]:
# convert True/False to 1/0 in order protocol dummies
order_protocol_dummies = order_protocol_dummies.astype(int)
order_protocol_dummies.head()

,order_protocol_1.0,order_protocol_2.0,order_protocol_3.0,order_protocol_4.0,order_protocol_5.0,order_protocol_6.0,order_protocol_7.0
0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0


In [21]:
# create dummies for market id
market_id_dummies = pd.get_dummies(historical_data.market_id)
market_id_dummies = market_id_dummies.add_prefix('market_id_')
market_id_dummies = market_id_dummies.astype(int)
market_id_dummies.head()

,market_id_1.0,market_id_2.0,market_id_3.0,market_id_4.0,market_id_5.0,market_id_6.0
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,0,0,1,0,0,0


In [22]:
# create dummies for store primary category
store_primary_category_dummies = pd.get_dummies(historical_data.nan_free_store_primary_category)
store_primary_category_dummies = store_primary_category_dummies.add_prefix('category_')
store_primary_category_dummies = store_primary_category_dummies.astype(int)
store_primary_category_dummies.head()

,category_afghan,category_african,category_alcohol,category_alcohol-plus-food,category_american,category_argentine,category_asian,category_barbecue,category_belgian,category_brazilian,...,category_southern,category_spanish,category_steak,category_sushi,category_tapas,category_thai,category_turkish,category_vegan,category_vegetarian,category_vietnamese
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We won't use the original columns as we transformed them. Also, we extract the total delivery duration from created and actual delivery time columns. Hence, we don't need them anymore. Let's drop.

In [23]:
# dropping unnecessary columns
train_df = historical_data.drop(columns=['created_at', 'market_id', 'store_id', 'store_primary_category', 'actual_delivery_time',
                                         'nan_free_store_primary_category', 'order_protocol'])
train_df.head()

,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,actual_total_delviery_duration,estimated_non_prep_duration,busy_dashers_ratio
0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,3779.0,1307.0,0.424242
1,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,4024.0,1136.0,2.000000
2,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0,1781.0,1136.0,0.000000
3,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0,3075.0,735.0,1.000000
4,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0,2390.0,1096.0,1.000000


Finally, concatenate all created columns and the final version of our DataFrame. We all have numerical values now. We can convert all to float to use them with machine learning methods.

In [25]:
train_df = pd.concat([train_df, order_protocol_dummies, market_id_dummies, store_primary_category_dummies], axis=1)

# align dtype over dataset
train_df = train_df.astype("float32")
train_df.head()

,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,...,category_southern,category_spanish,category_steak,category_sushi,category_tapas,category_thai,category_turkish,category_vegan,category_vegetarian,category_vietnamese
0,4.0,3441.0,4.0,557.0,1239.0,33.0,14.0,21.0,446.0,861.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1900.0,1.0,1400.0,1400.0,1.0,2.0,2.0,446.0,690.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1900.0,1.0,1900.0,1900.0,1.0,0.0,0.0,446.0,690.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.0,6900.0,5.0,600.0,1800.0,1.0,1.0,2.0,446.0,289.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,3900.0,3.0,1100.0,1600.0,6.0,6.0,9.0,446.0,650.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# checking the final dataset
train_df.describe()

,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,...,category_southern,category_spanish,category_steak,category_sushi,category_tapas,category_thai,category_turkish,category_vegan,category_vegetarian,category_vietnamese
count,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,181166.000000,181166.000000,181166.000000,197428.000000,196902.000000,...,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000,197428.000000
mean,3.196391,2682.331543,2.670791,686.218506,1159.588623,44.808094,41.739746,58.050064,308.560181,545.358948,...,0.000815,0.000187,0.005465,0.011300,0.000740,0.037254,0.001175,0.001433,0.004234,0.030928
std,2.666546,1823.093750,1.630255,522.038635,558.411377,34.526787,32.145733,52.661831,90.139648,219.352905,...,0.028545,0.013689,0.073725,0.105701,0.027184,0.189384,0.034260,0.037834,0.064935,0.173122
min,1.000000,0.000000,1.000000,-86.000000,0.000000,-4.000000,-5.000000,-6.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1400.000000,1.000000,299.000000,800.000000,17.000000,15.000000,17.000000,251.000000,382.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,2200.000000,2.000000,595.000000,1095.000000,37.000000,34.000000,41.000000,251.000000,544.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,3395.000000,3.000000,949.000000,1395.000000,65.000000,62.000000,85.000000,446.000000,702.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,411.000000,27100.000000,20.000000,14700.000000,14700.000000,171.000000,154.000000,285.000000,2715.000000,2088.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
train_df['busy_dashers_ratio'].describe()

count    1.775900e+05
mean              NaN
std               NaN
min              -inf
25%      8.269231e-01
50%      9.622642e-01
75%      1.000000e+00
max               inf
Name: busy_dashers_ratio, dtype: float64

It seems we have high max-min differences for some columns and infinity values. We need to replace all infinity values with nan to drop since they are not useful in the prediction.

In [28]:
# check infinte values using numpy isfinite() function (could also use isinf() function)
np.where(np.any(-np.isfinite(train_df), axis=0) == True) # show the rows with infinite values

(array([ 5,  6,  7,  9, 10, 11, 12]),)

In [29]:
# replace infinite values with NaN to drop all NaN values
train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# drop all NaN values
train_df.dropna(inplace=True)
train_df.shape

(177070, 100)

In [30]:
# confirm no more remaining NaN values
train_df.isna().sum().sum()

0

In [31]:
# export the final dataset
train_df.to_csv("data/train_df.csv", index=False)